In [29]:
import os
import time
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
from os import listdir
import pandas as pd
import numpy as np
import glob
import cv2
import json
from os.path import expanduser
import splitfolders
import shutil
from define_path import Def_Path

from tqdm import tqdm

import torch 
import torchvision
from torchvision import models
from torchvision.models.detection.rpn import AnchorGenerator
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import torchvision.transforms as T
from torchvision.transforms import functional as F
from torchsummary import summary
from sklearn.model_selection import train_test_split

import albumentations as A # Library for augmentations

import matplotlib.pyplot as plt 
from PIL import Image

import transforms, utils, engine, train
from utils import collate_fn
from engine import train_one_epoch, evaluate

t = torch.cuda.get_device_properties(0).total_memory
print(t)
torch.cuda.empty_cache()

r = torch.cuda.memory_reserved(0)
print(r)
a = torch.cuda.memory_allocated(0)
print(a)
# f = r-a  # free inside reserved

weights_path = '/home/jc-merlab/Pictures/Data/trained_models/keypointsrcnn_weights_sim_b1_e25_v0.pth'

10504699904
631242752
544849920


In [30]:
# to generalize home directory. User can change their parent path without entering their home directory
path = Def_Path()

parent_path =  path.home + "/Pictures/" + "Data/"

root_dir = parent_path + path.year + "-" + path.month + "-" + path.day + "/"

In [31]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.cuda.set_per_process_memory_fraction(0.9, 0)
print(device)

cuda


In [32]:
def train_transform():
    return A.Compose([
        A.Sequential([
            A.RandomRotate90(p=1), # Random rotation of an image by 90 degrees zero or more times
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.3, brightness_by_max=True, always_apply=False, p=1), # Random change of brightness & contrast
        ], p=1)
#         A.Resize(640, 480)  # Resize all images to be 640x480
    ],
    keypoint_params=A.KeypointParams(format='xy'), # More about keypoint formats used in albumentations library read at https://albumentations.ai/docs/getting_started/keypoints_augmentation/
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['bboxes_labels']) # Bboxes should have labels, read more at https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/
    )

In [33]:
def train_test_split(src_dir):
    dst_dir_img = src_dir + "images"
    dst_dir_anno = src_dir + "annotations"
    
    if os.path.exists(dst_dir_img) and os.path.exists(dst_dir_anno):
        print("folders exist")
    else:
        os.mkdir(dst_dir_img)
        os.mkdir(dst_dir_anno)
        
    for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
        shutil.copy(jpgfile, dst_dir_img)

    for jsonfile in glob.iglob(os.path.join(src_dir, "*.json")):
        shutil.copy(jsonfile, dst_dir_anno)
        
    output = parent_path + "split_folder_output" + "-" + path.year + "-" + path.month + "-" + path.day 
    
    splitfolders.ratio(src_dir, # The location of dataset
                   output=output, # The output location
                   seed=42, # The number of seed
                   ratio=(.7, .2, .1), # The ratio of split dataset
                   group_prefix=None, # If your dataset contains more than one file like ".jpg", ".pdf", etc
                   move=False # If you choose to move, turn this into True
                   )
    
    shutil.rmtree(dst_dir_img)
    shutil.rmtree(dst_dir_anno)
    
    return output  
    

In [34]:
class KPDataset(Dataset):
    def __init__(self, root, transform=None, demo=False):                
        self.root = root
        self.transform = transform
        self.demo = demo # Use demo=True if you need transformed and original images (for example, for visualization purposes)
        self.imgs_files = sorted(os.listdir(os.path.join(root, "images")))
        self.annotations_files = sorted(os.listdir(os.path.join(root, "annotations")))
    
    def __getitem__(self, idx):
        img_file = self.imgs_files[idx]
        img_path = os.path.join(self.root, "images", self.imgs_files[idx])
        annotations_path = os.path.join(self.root, "annotations", self.annotations_files[idx])

        img_original = cv2.imread(img_path)
        img_original = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)
        
        with open(annotations_path) as f:
            data = json.load(f)
            bboxes_original = data['bboxes']
            keypoints_original = data['keypoints']
            
            # All objects are keypoints on the robot
            bboxes_labels_original = [] 
            bboxes_labels_original.append('base_joint')
            bboxes_labels_original.append('joint2')
            bboxes_labels_original.append('joint3')
            bboxes_labels_original.append('joint4')
            bboxes_labels_original.append('joint5')
            bboxes_labels_original.append('joint6')  

        if self.transform:   
            # Converting keypoints from [x,y,visibility]-format to [x, y]-format + Flattening nested list of keypoints            
            # For example, if we have the following list of keypoints for three objects (each object has two keypoints):
            # [[obj1_kp1, obj1_kp2], [obj2_kp1, obj2_kp2], [obj3_kp1, obj3_kp2]], where each keypoint is in [x, y]-format            
            # Then we need to convert it to the following list:
            # [obj1_kp1, obj1_kp2, obj2_kp1, obj2_kp2, obj3_kp1, obj3_kp2]
            keypoints_original_flattened = [el[0:2] for kp in keypoints_original for el in kp]
            
            # Apply augmentations
            transformed = self.transform(image=img_original, bboxes=bboxes_original, bboxes_labels=bboxes_labels_original, keypoints=keypoints_original_flattened)
            img = transformed['image']
            bboxes = transformed['bboxes']
            # Unflattening list transformed['keypoints']
            # For example, if we have the following list of keypoints for three objects (each object has two keypoints):
            # [obj1_kp1, obj1_kp2, obj2_kp1, obj2_kp2, obj3_kp1, obj3_kp2], where each keypoint is in [x, y]-format
            # Then we need to convert it to the following list:
            # [[obj1_kp1, obj1_kp2], [obj2_kp1, obj2_kp2], [obj3_kp1, obj3_kp2]]
            keypoints_transformed_unflattened = np.reshape(np.array(transformed['keypoints']), (-1,1,2)).tolist()

            # Converting transformed keypoints from [x, y]-format to [x,y,visibility]-format by appending original visibilities to transformed coordinates of keypoints
            keypoints = []
            for o_idx, obj in enumerate(keypoints_transformed_unflattened):
#                 print("object", obj)
#                 print(" obj index", o_idx)# Iterating over objects
                obj_keypoints = []
                for k_idx, kp in enumerate(obj): # Iterating over keypoints in each object
                    obj_keypoints.append(kp + [keypoints_original[o_idx][k_idx][2]])
                keypoints.append(obj_keypoints)
        
        else:
            img, bboxes, keypoints = img_original, bboxes_original, keypoints_original        
        
        # Convert everything into a torch tensor        
        bboxes = torch.as_tensor(bboxes, dtype=torch.float32)       
        target = {}
        labels = [1, 2, 3, 4, 5, 6]            
        target["boxes"] = bboxes
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64) # all objects are joint positions
        target["image_id"] = torch.tensor([idx])
        target["area"] = (bboxes[:, 3] - bboxes[:, 1]) * (bboxes[:, 2] - bboxes[:, 0])
        target["iscrowd"] = torch.zeros(len(bboxes), dtype=torch.int64)
        target["keypoints"] = torch.as_tensor(keypoints, dtype=torch.float32)
        img = F.to_tensor(img)        
        bboxes_original = torch.as_tensor(bboxes_original, dtype=torch.float32)
        target_original = {}
        target_original["boxes"] = bboxes_original
        target_original["labels"] = torch.as_tensor(labels, dtype=torch.int64) # all objects are glue tubes
        target_original["image_id"] = torch.tensor([idx])
        target_original["area"] = (bboxes_original[:, 3] - bboxes_original[:, 1]) * (bboxes_original[:, 2] - bboxes_original[:, 0])
        target_original["iscrowd"] = torch.zeros(len(bboxes_original), dtype=torch.int64)
        target_original["keypoints"] = torch.as_tensor(keypoints_original, dtype=torch.float32)        
        img_original = F.to_tensor(img_original)

        if self.demo:
            return img, target, img_original, target_original, img_file
        else:
            return img, target, img_file
    
    def __len__(self):
        return len(self.imgs_files)

In [35]:
def get_model(num_keypoints, weights_path=None):
    
    anchor_generator = AnchorGenerator(sizes=(32, 64, 128, 256, 512), aspect_ratios=(0.25, 0.5, 0.75, 1.0, 2.0, 3.0, 4.0))
    model = torchvision.models.detection.keypointrcnn_resnet50_fpn(pretrained=False,
                                                                   pretrained_backbone=True,
                                                                   num_keypoints=num_keypoints,
                                                                   num_classes = 7, # Background is the first class, object is the second class
                                                                   rpn_anchor_generator=anchor_generator)

    if weights_path:
        state_dict = torch.load(weights_path)
        model.load_state_dict(state_dict)        
        
    return model

In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as func
import networkx as nx


class GNNEncoder(nn.Module):
    def __init__(self, vertices_dim=5, hidden_dim=128, num_vertices=6, num_edge_features=4):
        super(GNNEncoder, self).__init__()
        self.f_enc = nn.Linear(vertices_dim, hidden_dim)
        self.f_e1 = nn.Linear((hidden_dim * 2)+2, hidden_dim)
        self.f_v = nn.Linear(hidden_dim, hidden_dim)
        self.f_e2 = nn.Linear((hidden_dim * 2)+2, num_edge_features)
        self.num_vertices = num_vertices        
    
    def get_node_features(self, vertices):
#         print("Vertices in node features", vertices)
        node_features = []
        for keypoint in vertices:
            x, y, confidence, visibility, label = keypoint
            node_features.append([x, y, confidence, visibility, label])        
        nodes = torch.tensor(node_features, dtype=torch.float).to(device)
#         print(nodes)
        return nodes

    def get_edge_features(self, vertices):
        edges = [(0,1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0)]
        print(edges)
        edge_features = []
        for edge in edges:
            print(edge)
            print(edge[0])
            print(edge[1])
            k1, k2 = vertices[edge[0]][:2], vertices[edge[1]][:2]
            distance = torch.norm(k1 - k2)
            angle = torch.atan2(k2[1] - k1[1], k2[0] - k1[0])
            edge_features.append([distance.item(), angle.item()])
            
        edges = torch.tensor(edges, dtype=torch.long).to(device)
        edge_features = torch.tensor(edge_features, dtype=torch.float).to(device)
        return edges, edge_features

    def forward(self, vertices):
        nodes = self.get_node_features(vertices)
        edges, edge_features = self.get_edge_features(vertices)
        h1 = self.f_enc(nodes)
        h1_source = h1[edges[:, 0]]
        h1_target = h1[edges[:, 1]]
        h_e1 = self.f_e1(torch.cat((h1_source, h1_target, edge_features), dim=1))  # Include edge feature in the input
        h_j_2 = self.f_v(h_e1)
        h2_source = h_j_2[edges[:, 0]]
        h2_target = h_j_2[edges[:, 1]]
        h_e2 = self.f_e2(torch.cat((h1_source, h1_target, edge_features), dim=1))  # Include edge feature in the input
        h_e2_prob = torch.sigmoid(h_e2)
        return vertices, h_e2_prob, edges, edge_features

class GNNDecoder(nn.Module):
    def __init__(self, vertices_dim=5, hidden_dim=128, num_vertices=6, num_edge_features=4):
        super(GNNDecoder, self).__init__()
        self.f_e = nn.Linear((vertices_dim * 2)+2, num_edge_features)  # Concatenate two vertices features
        self.f_h = nn.Linear(num_edge_features, vertices_dim)  # Transform h_ij to the same dimension as vertices
        self.f_v = nn.Linear(vertices_dim, vertices_dim)  # Update vertex feature
    
    def forward(self, vertices, h_e2_prob, edges, edge_features):
        h_source = vertices[edges[:, 0]]
        h_target = vertices[edges[:, 1]]
        h = torch.zeros_like(vertices)

        for idx, (i, j) in enumerate(edges):  # Iterate over edges
            single_edge_features = edge_features[idx].unsqueeze(0)    
            h_ij = h_e2_prob[idx] * self.f_e(torch.cat((h_source[idx].unsqueeze(0), h_target[idx].unsqueeze(0), single_edge_features), dim=1))  # Include edge weights in the input
            h_ij_transformed = self.f_h(h_ij.squeeze())  # Transform h_ij to the same dimension as vertices
            h[j] += h_ij_transformed  # Accumulate edge features to the target vertex

        h_transformed = self.f_v(h.view(-1, vertices.shape[1]))  # Transform h
        h_transformed = h_transformed.view(vertices.shape)  # Reshape back to original shape
        vertices_g = vertices + h_transformed  # Update vertex features

        return vertices_g  # Return vertices_g as the prediction and vertices_g itself as the mean for Gaussian distribution



In [49]:
class OccludedKeyPointLoss(nn.Module):
    def __init__(self, delta=1.0):
        super().__init__()
        self.delta = delta

    def forward(self, vertices_pred, vertices_gt):
        vertices_pred = vertices_pred[:, :3]  # Considering only x, y coordinates, confidence_score
        vertices_gt = vertices_gt[:, :3]  # Considering only x, y coordinates, confidence_score
        # Compute differences
        diff = (vertices_gt - vertices_pred).abs()
        # Compute Huber loss
        huber_loss = torch.where(diff < self.delta, 0.5 * diff**2, self.delta * (diff - 0.5 * self.delta))
        return huber_loss.mean()
    
def visibility_loss (vertices_pred, vertices_gt):    
    return func.cross_entropy(vertices_pred[:, 3], vertices_gt[:, 3])  # Loss based on visibility of keypoints

def edge_loss(edges_prob, edges_gt):
    # Compute the cross-entropy loss
    loss = -torch.sum(edges_gt.to(device) * torch.log(torch.clamp(edges_prob, min=1e-7)))                      
    return loss

def temporal_consistency_loss(y_true_sequence, y_pred_sequence):
    loss = 0
    for t in range(1, len(y_true_sequence)):
        # Selecting the x, y coordinates and visibility for true and predicted sequences
        true_diff = y_true_sequence[t, :, :] - y_true_sequence[t-1, :, :]
        pred_diff = y_pred_sequence[t, :, :] - y_pred_sequence[t-1, :, :]
        loss += torch.mean(torch.abs(true_diff - pred_diff))
    return loss

In [53]:
class KeypointPipeline(nn.Module):
    def __init__(self, weights_path, num_vertices):
        super().__init__()

        self.keypoint_model = torch.load(weights_path).to(device)
        self.num_vertices = num_vertices
        self.gnn_encoder = GNNEncoder()
        self.gnn_decoder = GNNDecoder()

    def process_model_output(self, output):
        scores = output[0]['scores'].detach().cpu().numpy()
        high_scores_idxs = np.where(scores > 0.7)[0].tolist()

        post_nms_idxs = torchvision.ops.nms(output[0]['boxes'][high_scores_idxs], 
                                            output[0]['scores'][high_scores_idxs], 0.3).cpu().numpy()

        confidence = output[0]['scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()
        labels = output[0]['labels'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()
        keypoints = []
        for idx, kps in enumerate(output[0]['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()):
            # Setting t_i = 1 because label is found
            keypoints.append(list(map(int, kps[0,0:2])) + [confidence[idx]] + [1] + [labels[idx]])

        # Create a dictionary where the key is the label and the value is the keypoint
        label_to_keypoint = {}
        for keypoint in keypoints:
            label = keypoint[-1]
            if label not in label_to_keypoint or label_to_keypoint[label][-2] < keypoint[-2]:
                label_to_keypoint[label] = keypoint
        keypoints = [value for key, value in sorted(label_to_keypoint.items())]
        
        print(keypoints)
                
        keypoints = torch.stack([torch.tensor(kp) for kp in keypoints]).float().to(device)
        visibility = keypoints[:, 3].unsqueeze(1)  # Extracting the visibility
        keypoints_visible = keypoints * visibility  # Predicted visible vertices
        vertices, self.enc_e, self.edges, self.edge_features = self.gnn_encoder(keypoints_visible)  # Adjust here to include edge weights
        vertices_pred = self.gnn_decoder(vertices, self.enc_e, self.edges, self.edge_features)  # Adjust here to pass edge weights
        return vertices_pred

    def process_image(self, img):
        img = img.unsqueeze(0).to(device)
        # Temporarily set the keypoint model to evaluation mode
        keypoint_model_training = self.keypoint_model.training  # Save the current mode
        self.keypoint_model.eval()
        with torch.no_grad():
            output = self.keypoint_model(img)
        # Set the keypoint model back to its previous mode
        self.keypoint_model.train(keypoint_model_training)
        img = (img[0].permute(1,2,0).detach().cpu().numpy() * 255).astype(np.uint8)
        labeled_keypoints = self.process_model_output(output)

        return labeled_keypoints

    def forward(self, imgs):
        outputs = []

        for i in range(imgs.shape[0]):
            labeled_keypoints = self.process_image(imgs[i])
            outputs.append(labeled_keypoints)
            
        print(outputs)

        return outputs
    

In [54]:
# Define the model
model = KeypointPipeline(weights_path, num_vertices=6)
model = model.to(device)

# Define the loss
criterion = OccludedKeyPointLoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1  # Define your number of epochs
batch_size = 1

KEYPOINTS_FOLDER_TRAIN = train_test_split(root_dir) +"/train" #train_test_split(root_dir) +"/train"
KEYPOINTS_FOLDER_VAL = train_test_split(root_dir) +"/val"
KEYPOINTS_FOLDER_TEST = train_test_split(root_dir) +"/test"

dataset_train = KPDataset(KEYPOINTS_FOLDER_TRAIN, transform=None, demo=False)
dataset_val = KPDataset(KEYPOINTS_FOLDER_VAL, transform=None, demo=False)
dataset_test = KPDataset(KEYPOINTS_FOLDER_TEST, transform=None, demo=False)

data_loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
data_loader_val = DataLoader(dataset_val, batch_size=1, shuffle=False, collate_fn=collate_fn)
data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, collate_fn=collate_fn)

v = 7

# Initialize sequences for true and predicted keypoints
y_true_sequence = []
y_pred_sequence = []

model.train()
for epoch in range(num_epochs):
    start_time = time.time()
    for i, batch in enumerate(data_loader_train):
        img_tuple, target_dict_tuple, img_files = batch
        print(f"Processing batch {i+1} with images:", img_files)
        
        imgs = [img.to(device) for img in img_tuple]  # Create list of images

        # Process each image individually
        losses = []
        for i in range(len(imgs)):
            img = imgs[i].unsqueeze(0)  # Unsqueeze to add batch dimension

            # Prepare ground truth vertices for the image
            keypoints = target_dict_tuple[i]['keypoints'].to(device)
            print(keypoints.shape)
            # add visibility to ground truth
            visibility = torch.ones((keypoints.shape[0], keypoints.shape[1], 1)).to(device)
            print(visibility.shape)
            # add labels to ground truth
            labels = torch.arange(1, keypoints.shape[0] + 1, device=keypoints.device).view(-1, 1, 1).float()
            print(labels.shape)
            # update gt with vis and labels
            vertices_gt = torch.cat((keypoints, visibility, labels), dim=2).unsqueeze(0)  # Unsqueeze to add batch dimension
            
            vertices_gt = vertices_gt.squeeze()    
            print("Ground truth keypoints from annotations", vertices_gt)
            y_true_sequence.append(vertices_gt)

            # Forward pass
            output = model(img)
            vertices_pred = output[0]
            y_pred_sequence.append(vertices_pred)
            
            edges_prob = model.enc_e
            edges = model.edges
            edge_features = model.edge_features
            edges_gt = torch.cat((edges, edge_features), dim=1) 

            # Compute loss for the image
            huber_loss = criterion(vertices_pred, vertices_gt)
            ce_loss = edge_loss(edges_prob, edges_gt)
            vis_loss = visibility_loss(vertices_pred, vertices_gt)

            loss = huber_loss + ce_loss + vis_loss
            losses.append(loss)  # Store loss for the image
            
        # Convert true and predicted sequences to tensors
        y_true_tensor = torch.stack(y_true_sequence)
        y_pred_tensor = torch.stack(y_pred_sequence)
        
        # Compute temporal consistency loss
        temporal_loss = temporal_consistency_loss(y_true_tensor, y_pred_tensor)

        # Average loss over all images in the batch
        other_losses = torch.mean(torch.stack(losses))
        
        # Combine temporal loss with other losses
        total_loss = other_losses + temporal_loss

        # Backward pass and optimization
        optimizer.zero_grad()
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        # Clear the sequences for the next batch
        y_true_sequence.clear()
        y_pred_sequence.clear()

    end_time = time.time()
    epoch_time = end_time - start_time
    eta = epoch_time * (num_epochs - epoch - 1)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, ETA: {eta} seconds')

model_save_path = f"/home/jc-merlab/Pictures/Data/trained_models/keypointsrcnn_weights_occ_b{batch_size}_e{num_epochs}_v{v}.pth"

torch.save(model, model_save_path)
    
# Save the state dict of the model, not the entire model
# torch.save(model.state_dict(), model_save_path)
    
torch.save(model, model_save_path)



Copying files: 2662 files [00:00, 17665.48 files/s]
Copying files: 2662 files [00:00, 18266.00 files/s]
Copying files: 2662 files [00:00, 18070.47 files/s]


Processing batch 1 with images: ('001283.rgb.jpg',)
torch.Size([6, 1, 3])
torch.Size([6, 1, 1])
torch.Size([6, 1, 1])
Ground truth keypoints from annotations tensor([[257.9522, 366.9199,   1.0000,   1.0000,   1.0000],
        [257.9597, 283.0131,   1.0000,   1.0000,   2.0000],
        [310.8167, 223.4985,   1.0000,   1.0000,   3.0000],
        [326.3398, 237.2794,   1.0000,   1.0000,   4.0000],
        [421.4327, 263.5889,   1.0000,   1.0000,   5.0000],
        [437.2448, 277.0866,   1.0000,   1.0000,   6.0000]], device='cuda:0')
[[258, 367, 0.999961, 1, 1], [258, 283, 0.9999279, 1, 2], [310, 223, 0.9994629, 1, 3], [326, 237, 0.99981004, 1, 4], [420, 263, 0.9973404, 1, 5], [436, 277, 0.99862075, 1, 6]]
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0)]
(0, 1)
0
1
(1, 2)
1
2
(2, 3)
2
3
(3, 4)
3
4
(4, 5)
4
5
(5, 0)
5
0
[tensor([[ 2.2773e+02,  3.8220e+02, -2.2868e+01, -2.2048e+01, -3.0335e+01],
        [ 2.2409e+02,  3.0247e+02, -2.3551e+01, -2.5087e+01, -3.4491e+01],
        [ 3.0660e+02, 

[[258, 367, 0.9999609, 1, 1], [258, 283, 0.99985063, 1, 2], [180, 298, 0.9994709, 1, 3], [175, 278, 0.99977666, 1, 4], [129, 190, 0.99518293, 1, 5], [136, 170, 0.9946701, 1, 6]]
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0)]
(0, 1)
0
1
(1, 2)
1
2
(2, 3)
2
3
(3, 4)
3
4
(4, 5)
4
5
(5, 0)
5
0
[tensor([[258.0740, 380.7454, -10.9107,  -4.3629, -29.7017],
        [250.9534, 287.0386,  -7.9361,  -8.0972, -19.6656],
        [172.6104, 301.0805,  -7.0316,  -8.2983, -17.7715],
        [169.3872, 282.7172,  -7.1957,  -7.0262, -17.1281],
        [121.8588, 190.8902,  -2.8067,  -8.5147, -14.4601],
        [133.1175, 173.3974,  -4.6523,  -3.3981,  -6.3706]], device='cuda:0',
       grad_fn=<AddBackward0>)]
Processing batch 8 with images: ('000653.rgb.jpg',)
torch.Size([6, 1, 3])
torch.Size([6, 1, 1])
torch.Size([6, 1, 1])
Ground truth keypoints from annotations tensor([[257.9522, 366.9199,   1.0000,   1.0000,   1.0000],
        [257.9597, 283.0131,   1.0000,   1.0000,   2.0000],
        [232.4796,

Ground truth keypoints from annotations tensor([[257.9522, 366.9199,   1.0000,   1.0000,   1.0000],
        [257.9597, 283.0131,   1.0000,   1.0000,   2.0000],
        [311.1267, 223.7719,   1.0000,   1.0000,   3.0000],
        [326.5832, 237.6358,   1.0000,   1.0000,   4.0000],
        [349.7068, 333.8601,   1.0000,   1.0000,   5.0000],
        [336.2486, 344.2185,   1.0000,   1.0000,   6.0000]], device='cuda:0')
[[258, 367, 0.99995553, 1, 1], [258, 283, 0.9999269, 1, 2], [311, 224, 0.9993617, 1, 3], [326, 238, 0.9997671, 1, 4], [349, 333, 0.994768, 1, 5], [335, 345, 0.992649, 1, 6]]
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0)]
(0, 1)
0
1
(1, 2)
1
2
(2, 3)
2
3
(3, 4)
3
4
(4, 5)
4
5
(5, 0)
5
0
[tensor([[ 2.5202e+02,  3.6968e+02, -7.3676e+00, -7.5751e+00, -2.0413e+01],
        [ 2.5293e+02,  2.8559e+02, -5.8195e+00, -6.8862e+00, -1.8481e+01],
        [ 3.0901e+02,  2.2789e+02, -6.8384e+00, -2.2871e+00, -8.0704e+00],
        [ 3.2482e+02,  2.4192e+02, -8.6406e+00,  1.9753e-01,  1.751

[[258, 367, 0.99996126, 1, 1], [258, 283, 0.9998666, 1, 2], [180, 298, 0.9988061, 1, 3], [175, 278, 0.9996946, 1, 4], [80, 251, 0.9971945, 1, 5], [59, 241, 0.99177974, 1, 6]]
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0)]
(0, 1)
0
1
(1, 2)
1
2
(2, 3)
2
3
(3, 4)
3
4
(4, 5)
4
5
(5, 0)
5
0
[tensor([[ 2.6250e+02,  3.7878e+02, -5.5615e+00, -2.6247e+00, -3.4002e+01],
        [ 2.5530e+02,  2.8383e+02, -2.9965e+00, -5.3168e+00, -1.6956e+01],
        [ 1.7692e+02,  2.9818e+02, -2.6127e+00, -5.5410e+00, -1.5383e+01],
        [ 1.7337e+02,  2.8015e+02, -3.3428e+00, -4.4305e+00, -1.5035e+01],
        [ 7.6533e+01,  2.5011e+02,  2.0365e-01, -7.1159e+00, -1.8007e+01],
        [ 5.7616e+01,  2.4303e+02, -1.3297e+00, -5.2572e+00, -1.6963e+01]],
       device='cuda:0', grad_fn=<AddBackward0>)]
Processing batch 24 with images: ('001063.rgb.jpg',)
torch.Size([6, 1, 3])
torch.Size([6, 1, 1])
torch.Size([6, 1, 1])
Ground truth keypoints from annotations tensor([[257.9522, 366.9199,   1.0000,   1.0000,  

Processing batch 30 with images: ('000046.rgb.jpg',)
torch.Size([6, 1, 3])
torch.Size([6, 1, 1])
torch.Size([6, 1, 1])
Ground truth keypoints from annotations tensor([[257.9522, 366.9199,   1.0000,   1.0000,   1.0000],
        [257.9597, 283.0131,   1.0000,   1.0000,   2.0000],
        [179.6374, 297.7307,   1.0000,   1.0000,   3.0000],
        [175.7790, 277.2977,   1.0000,   1.0000,   4.0000],
        [154.8367, 180.4464,   1.0000,   1.0000,   5.0000],
        [175.5857, 172.2121,   1.0000,   1.0000,   6.0000]], device='cuda:0')
[[258, 367, 0.9999621, 1, 1], [258, 283, 0.9998272, 1, 2], [180, 297, 0.999373, 1, 3], [176, 277, 0.9997309, 1, 4], [155, 180, 0.9949458, 1, 5], [175, 171, 0.99484247, 1, 6]]
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0)]
(0, 1)
0
1
(1, 2)
1
2
(2, 3)
2
3
(3, 4)
3
4
(4, 5)
4
5
(5, 0)
5
0
[tensor([[261.8105, 374.3018,  -3.1466,  -1.1289, -23.5199],
        [256.7708, 282.5891,  -0.7917,  -4.2992, -16.0163],
        [178.4125, 296.0361,  -0.6207,  -4.5109, -14

[[258, 367, 0.99996126, 1, 1], [257, 283, 0.99992573, 1, 2], [194, 235, 0.99982005, 1, 3], [206, 218, 0.99961394, 1, 4], [264, 136, 0.9972197, 1, 5], [281, 124, 0.9978126, 1, 6]]
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0)]
(0, 1)
0
1
(1, 2)
1
2
(2, 3)
2
3
(3, 4)
3
4
(4, 5)
4
5
(5, 0)
5
0
[tensor([[ 2.6082e+02,  3.6981e+02,  8.2480e-01, -8.4401e-01, -1.6955e+01],
        [ 2.5755e+02,  2.8191e+02,  1.3462e+00, -2.9449e+00, -1.5374e+01],
        [ 1.9362e+02,  2.3247e+02,  2.0733e+00, -2.7885e+00, -9.6192e+00],
        [ 2.0668e+02,  2.1830e+02, -1.0097e-01, -1.0151e+00, -5.5956e+00],
        [ 2.6426e+02,  1.3540e+02,  1.4778e+00, -3.1867e-01,  3.9787e-01],
        [ 2.7991e+02,  1.2267e+02, -2.0463e-01,  9.0756e-01,  1.2460e+01]],
       device='cuda:0', grad_fn=<AddBackward0>)]
Processing batch 40 with images: ('000523.rgb.jpg',)
torch.Size([6, 1, 3])
torch.Size([6, 1, 1])
torch.Size([6, 1, 1])
Ground truth keypoints from annotations tensor([[257.9522, 366.9199,   1.0000,   1.000

Processing batch 46 with images: ('001002.rgb.jpg',)
torch.Size([6, 1, 3])
torch.Size([6, 1, 1])
torch.Size([6, 1, 1])
Ground truth keypoints from annotations tensor([[257.9522, 366.9199,   1.0000,   1.0000,   1.0000],
        [257.9597, 283.0131,   1.0000,   1.0000,   2.0000],
        [281.4428, 206.9587,   1.0000,   1.0000,   3.0000],
        [301.2842, 213.0685,   1.0000,   1.0000,   4.0000],
        [385.5836, 265.1099,   1.0000,   1.0000,   5.0000],
        [392.7202, 245.2632,   1.0000,   1.0000,   6.0000]], device='cuda:0')
[[258, 367, 0.9999609, 1, 1], [258, 283, 0.9999275, 1, 2], [281, 206, 0.9998609, 1, 3], [301, 213, 0.9998411, 1, 4], [384, 265, 0.9955634, 1, 5], [392, 246, 0.99504787, 1, 6]]
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0)]
(0, 1)
0
1
(1, 2)
1
2
(2, 3)
2
3
(3, 4)
3
4
(4, 5)
4
5
(5, 0)
5
0
[tensor([[ 2.5881e+02,  3.6478e+02,  2.7179e+00, -2.0513e+00, -1.4064e+01],
        [ 2.5963e+02,  2.8246e+02,  1.5254e+00, -1.6757e+00, -1.5266e+01],
        [ 2.8153e+02,

[tensor([[ 2.6074e+02,  3.6886e+02,  1.9717e+00,  4.0501e-01, -1.4785e+01],
        [ 2.6014e+02,  2.8342e+02,  6.3922e-01, -7.3730e-01, -1.5436e+01],
        [ 2.8389e+02,  2.0659e+02,  9.8111e-01, -2.0249e-01, -4.8515e+00],
        [ 3.0207e+02,  2.1275e+02, -4.1652e-01,  2.6145e-01,  3.5594e+00],
        [ 3.3093e+02,  1.1631e+02,  2.1921e+00, -9.2089e-03,  8.8031e+00],
        [ 3.2843e+02,  9.3425e+01,  1.1422e+00,  2.6660e-01,  2.0036e+01]],
       device='cuda:0', grad_fn=<AddBackward0>)]
Processing batch 53 with images: ('000702.rgb.jpg',)
torch.Size([6, 1, 3])
torch.Size([6, 1, 1])
torch.Size([6, 1, 1])
Ground truth keypoints from annotations tensor([[257.9522, 366.9199,   1.0000,   1.0000,   1.0000],
        [257.9597, 283.0131,   1.0000,   1.0000,   2.0000],
        [233.1790, 207.3317,   1.0000,   1.0000,   3.0000],
        [252.9274, 200.8611,   1.0000,   1.0000,   4.0000],
        [349.1352, 224.3284,   1.0000,   1.0000,   5.0000],
        [360.4466, 242.6084,   1.0000,  

KeyboardInterrupt: 

In [ ]:
def visualize_and_save(img, vertices, filename):
    print("type of image befor conversion",type(img))    
    print("type of vertices before conversion", type(vertices))
    print(img)
    img = (img.permute(1,2,0).cpu().numpy() * 255).astype(np.uint8)
#     img = (img * 255).astype(np.uint8)  # Convert back from [0, 1] range to [0, 255]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    vertices = vertices.cpu().numpy()

    print(f"Image shape before saving: {img.shape}")  # print the image shape
    print("type of vertices", type(vertices))
#     print("entered vertices", vertices)
#     print("entered image", img)

    # Convert grayscale to BGR if necessary
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        
    for i in range(vertices.shape[0]):
        img = cv2.circle(img, (int(vertices[i, 0]), int(vertices[i, 1])), radius=2, color=(0, 0, 255), thickness=-1)
        
    result = cv2.imwrite(filename, img)
    print(f"Image saved at {filename}: {result}")  # print if save was successful

    # If the image didn't save correctly, save the image data to a text file for examination
    if not result:
        with open(filename + ".txt", "w") as f:
            np.savetxt(f, img.flatten())

In [14]:
def test_and_save_model(model, data_loader_test):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    total_trifocal_loss = 0.0
    total_ce_loss = 0.0
    total_vis_loss = 0.0
    num_batches = 0

    y_true_sequence = []
    y_pred_sequence = []

    # We don't need to track gradients during evaluation
    with torch.no_grad():
        for idx, batch in enumerate(data_loader_test):
            img_tuple, target_dict_tuple, img_files = batch

            total_batch_loss = 0.0
            total_batch_trifocal_loss = 0.0
            total_batch_ce_loss = 0.0
            total_batch_vis_loss = 0.0

            # Process each image individually
            for i in range(len(img_tuple)):
                img = img_tuple[i].to(device)
                target = target_dict_tuple[i]

                # Prepare ground truth vertices for the image
                keypoints = target['keypoints'].to(device)
                visibility = torch.ones((keypoints.shape[0], keypoints.shape[1], 1)).to(device)
                vertices_gt = torch.cat((keypoints, visibility), dim=2).unsqueeze(0)  # Unsqueeze to add batch dimension
                vertices_gt = vertices_gt.squeeze()
                y_true_sequence.append(vertices_gt)

                # Forward pass
                output = model(img.unsqueeze(0))
                vertices_pred = output[0]
                y_pred_sequence.append(vertices_pred)

                edges_prob = model.enc_e
                edges = model.edges
                edge_features = model.edge_features
                edges_gt = torch.cat((edges, edge_features), dim=1) 

                trifocal_loss = criterion(vertices_pred, vertices_gt)
                ce_loss = edge_loss(edges_prob, edges_gt)
                vis_loss = visibility_loss(vertices_pred, vertices_gt)
                loss = trifocal_loss + ce_loss + vis_loss

                total_batch_loss += loss.item()
                total_batch_trifocal_loss += trifocal_loss.item()
                total_batch_ce_loss += ce_loss.item()
                total_batch_vis_loss += vis_loss.item()

                # Visualize and save the prediction
                filename = f'/home/jc-merlab/Pictures/Data/occ_vis_data/image_{idx}_{i}.jpg'
                visualize_and_save(img, vertices_pred, filename)
                print(f"Image saved at {filename}")  # Print statement to confirm image save

            # Convert true and predicted sequences to tensors
            y_true_tensor = torch.stack(y_true_sequence)
            y_pred_tensor = torch.stack(y_pred_sequence)

            # Compute temporal consistency loss
            temporal_loss = temporal_consistency_loss(y_true_tensor, y_pred_tensor)

            total_loss += (total_batch_loss + temporal_loss) / len(img_tuple)
            total_trifocal_loss += total_batch_trifocal_loss / len(img_tuple)
            total_ce_loss += total_batch_ce_loss / len(img_tuple)
            num_batches += 1

            # Clear the sequences for the next batch
            y_true_sequence.clear()
            y_pred_sequence.clear()
    
    # Average the loss over all batches
    avg_loss = total_loss / num_batches
    avg_trifocal_loss = total_trifocal_loss / num_batches
    avg_ce_loss = total_ce_loss / num_batches
    
    print(f'Avg. Test Loss: {avg_loss}, Avg. Trifocal Loss: {avg_trifocal_loss}, Avg. Cross Entropy Loss: {avg_ce_loss}')
    return avg_loss, avg_trifocal_loss, avg_ce_loss

In [15]:
# avg_loss, avg_trifocal_loss, avg_ce_loss, all_preds = test_and_save_model(model, data_loader_test)

avg_loss, avg_trifocal_loss, avg_ce_loss = test_and_save_model(model, data_loader_test)

[tensor([[258.1160, 367.1554,   1.1444,   1.4266,   0.7307],
        [258.1682, 283.1862,   1.0715,   1.4331,   1.8153],
        [179.2048, 298.2015,   1.0242,   1.4298,   2.8854],
        [176.2362, 278.2211,   0.9826,   1.4296,   3.9374],
        [230.2745, 195.2383,   0.9325,   1.4263,   5.0094],
        [249.3097, 203.2556,   0.8869,   1.4251,   6.0728]], device='cuda:0')]
type of image befor conversion <class 'torch.Tensor'>
type of vertices before conversion <class 'torch.Tensor'>
tensor([[[0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         ...,
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980]],

        [[0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..

[tensor([[258.1219, 367.1616,   1.1376,   1.4237,   0.7398],
        [258.1682, 283.1862,   1.0715,   1.4331,   1.8153],
        [180.2048, 298.2015,   1.0242,   1.4298,   2.8853],
        [176.2362, 277.2211,   0.9827,   1.4296,   3.9375],
        [ 82.2674, 247.2315,   0.8957,   1.4269,   4.9996],
        [ 91.3055, 229.2555,   0.8881,   1.4253,   6.0634]], device='cuda:0')]
type of image befor conversion <class 'torch.Tensor'>
type of vertices before conversion <class 'torch.Tensor'>
tensor([[[0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         ...,
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980]],

        [[0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..

type of image befor conversion <class 'torch.Tensor'>
type of vertices before conversion <class 'torch.Tensor'>
tensor([[[0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         ...,
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980]],

        [[0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         ...,
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980]],

        [[0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0

[tensor([[258.1256, 367.1582,   1.1344,   1.4215,   0.7527],
        [258.1682, 283.1862,   1.0715,   1.4331,   1.8153],
        [178.2032, 276.2004,   1.0269,   1.4298,   2.8828],
        [180.2378, 255.2218,   0.9810,   1.4289,   3.9407],
        [183.2734, 157.2366,   0.9328,   1.4261,   5.0087],
        [190.3107, 136.2560,   0.8855,   1.4237,   6.0755]], device='cuda:0')]
type of image befor conversion <class 'torch.Tensor'>
type of vertices before conversion <class 'torch.Tensor'>
tensor([[[0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         ...,
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980]],

        [[0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..

[tensor([[258.1175, 367.1543,   1.1432,   1.4257,   0.7356],
        [258.1682, 283.1862,   1.0715,   1.4331,   1.8153],
        [178.2032, 277.2004,   1.0268,   1.4298,   2.8828],
        [179.2377, 256.2217,   0.9811,   1.4290,   3.9405],
        [252.2768, 189.2394,   0.9296,   1.4255,   5.0139],
        [273.3097, 181.2553,   0.8836,   1.4252,   6.0731]], device='cuda:0')]
type of image befor conversion <class 'torch.Tensor'>
type of vertices before conversion <class 'torch.Tensor'>
tensor([[[0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         ...,
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980]],

        [[0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..

...

[tensor([[258.1014, 367.1480,   1.1602,   1.4341,   0.7006],
        [258.1682, 283.1862,   1.0716,   1.4331,   1.8153],
        [296.2075, 213.2053,   1.0209,   1.4295,   2.8879],
        [314.2430, 223.2228,   0.9746,   1.4281,   3.9522],
        [365.2802, 307.2382,   0.9220,   1.4252,   5.0229],
        [373.3018, 328.2511,   0.8514,   1.4298,   6.0569]], device='cuda:0')]
type of image befor conversion <class 'torch.Tensor'>
type of vertices before conversion <class 'torch.Tensor'>
tensor([[[0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         ...,
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980]],

        [[0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..

[tensor([[258.0998, 367.1500,   1.1615,   1.4352,   0.6944],
        [257.1681, 283.1861,   1.0716,   1.4331,   1.8152],
        [311.2084, 223.2060,   1.0196,   1.4293,   2.8893],
        [326.2425, 237.2224,   0.9752,   1.4286,   3.9510],
        [350.2787, 334.2368,   0.9276,   1.4258,   5.0206],
        [338.3007, 348.2508,   0.8923,   1.4307,   6.0541]], device='cuda:0')]
type of image befor conversion <class 'torch.Tensor'>
type of vertices before conversion <class 'torch.Tensor'>
tensor([[[0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         ...,
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980]],

        [[0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..

[tensor([[258.1093, 367.1457,   1.1529,   1.4293,   0.7236],
        [258.1682, 283.1862,   1.0716,   1.4331,   1.8153],
        [311.2083, 223.2059,   1.0196,   1.4294,   2.8892],
        [326.2425, 238.2224,   0.9752,   1.4285,   3.9511],
        [423.2786, 214.2388,   0.8935,   1.4261,   5.0182],
        [422.3083, 230.2528,   0.8843,   1.4279,   6.0708]], device='cuda:0')]
type of image befor conversion <class 'torch.Tensor'>
type of vertices before conversion <class 'torch.Tensor'>
tensor([[[0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         ...,
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980]],

        [[0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..

[tensor([[258.1118, 367.1459,   1.1504,   1.4280,   0.7297],
        [258.1682, 283.1862,   1.0716,   1.4331,   1.8153],
        [311.2083, 224.2059,   1.0192,   1.4294,   2.8891],
        [327.2425, 238.2224,   0.9752,   1.4286,   3.9510],
        [413.2783, 189.2386,   0.9107,   1.4263,   5.0176],
        [421.3097, 204.2535,   0.8778,   1.4270,   6.0740]], device='cuda:0')]
type of image befor conversion <class 'torch.Tensor'>
type of vertices before conversion <class 'torch.Tensor'>
tensor([[[0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         ...,
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980]],

        [[0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..

[tensor([[258.1162, 367.1465,   1.1458,   1.4255,   0.7407],
        [258.1682, 282.1862,   1.0716,   1.4330,   1.8154],
        [311.2083, 224.2060,   1.0196,   1.4294,   2.8892],
        [327.2425, 238.2224,   0.9752,   1.4286,   3.9510],
        [397.2778, 170.2382,   0.9268,   1.4266,   5.0168],
        [416.3107, 158.2541,   0.8791,   1.4262,   6.0761]], device='cuda:0')]
type of image befor conversion <class 'torch.Tensor'>
type of vertices before conversion <class 'torch.Tensor'>
tensor([[[0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         ...,
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980]],

        [[0.6980, 0.6980, 0.6980,  ..., 0.6980, 0.6980, 0.6980],
         [0.6980, 0.6980, 0.6980,  ..

In [ ]:
import cv2
import os

# Directory containing images
dir_path = '/home/jc-merlab/Pictures/Data/occ_vis_data/'
images = []

# Ensure the images are sorted by name
for f in sorted(os.listdir(dir_path)):
    if f.endswith('.jpg') or f.endswith('.png'):  # Check for image file extension
        images.append(f)

# Determine the width and height from the first image
image_path = os.path.join(dir_path, images[0])
frame = cv2.imread(image_path)
cv2.imshow('video',frame)
height, width, channels = frame.shape

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Be sure to use the correct codec
video_filename = 'output.mp4'
video = cv2.VideoWriter(video_filename, fourcc, 3.0, (width, height))

for image in images:
    image_path = os.path.join(dir_path, image)
    frame = cv2.imread(image_path)
    video.write(frame)  # Write out frame to video

# Release everything when job is finished
video.release()
cv2.destroyAllWindows()

print("The output video is", video_filename)

In [55]:
model_path = '/home/jc-merlab/Pictures/Data/trained_models/keypointsrcnn_weights_occ_b16_e25_v6.pth'

model = torch.load(model_path).to(device)


image = Image.open("/home/jc-merlab/Pictures/Data/occluded_results_mi20_ma80_n2/occluded_000041.rgb.jpg")
print(type(image))

img = F.to_tensor(image).to(device)
img.unsqueeze_(0)
# print(image.shape)
# image = list(image)
# print(type(images))
# images = list(image.to(device) for image in images)

with torch.no_grad():
    model.to(device)
    model.eval()
    output = model(img)
    
keypoints = output[0]

print(keypoints)
plt.imshow(image)

# Assuming each keypoint is a tensor representing (x, y)
for i, keypoint in enumerate(keypoints):
    print(f'Key point {i}: {keypoint}')
    keypoint = keypoint.cpu().numpy()
    plt.plot(keypoint[0], keypoint[1], 'ro')
plt.show()

# Plotting the image

# plt.imshow(image)

# for keypoint in output[0]:
#     plt.plot(keypoint[0], keypoint[1], 'ro')

# plt.show()

<class 'PIL.JpegImagePlugin.JpegImageFile'>
[[258, 367, 0.9999529, 1, 1], [168, 278, 0.753524, 1, 4], [181, 178, 0.9980096, 1, 5], [203, 174, 0.99722606, 1, 6]]
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0)]
(0, 1)
0
1
(1, 2)
1
2
(2, 3)
2
3
(3, 4)
3
4


IndexError: index 4 is out of bounds for dimension 0 with size 4